### **Initialization**

In [0]:
# Importing Libraries
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

### **Reading From Bronze Layer**

In [0]:
df = spark.table("workspace.bronze.crm_cust_info")

## **Data Transformation**

### 1-Trim the strings

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))


### 2-Normalisation of data for martial_satus and gender

In [0]:
df = (
    df
    .withColumn(
        "cst_marital_status",
        F.when(F.upper(F.col("cst_marital_status")) == "S", "Single")
        .when(F.upper(F.col("cst_marital_status")) == "M", "Married")
        .otherwise("Unknown")
        )
    
    .withColumn(
        "cst_gndr",
        F.when(F.upper(F.col("cst_gndr")) == "M", "Male")
        .when(F.upper(F.col("cst_gndr")) == "F", "Female")
        .otherwise("Unknown")
        )
    )

### 3-Renaming Columns

In [0]:
# Creating new column names in a dictionary
RENAME_MAP = {
    "cst_id": "customer_id",
    "cst_key": "customer_number",
    "cst_firstname": "firstname",
    "cst_lastname": "lastname",
    "cst_marital_status": "marital_status",
    "cst_gndr": "gender",
    "cst_create_date": "created_date"
}

# Looping for all columns names and rename them
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

### 4-Removing data with null customer_id

In [0]:
df = df.filter(col("customer_id").isNotNull())

## **Write Into Silver Layer**

In [0]:
df.write.mode("overwrite").saveAsTable("workspace.silver.crm_customer")

## **Sanity check of Data Frame**

In [0]:
df.display()